In [ ]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


In [ ]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)

In [ ]:
image,label=train_data[0]

In [ ]:
image

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [ ]:
image.size()

torch.Size([3, 32, 32])

In [ ]:
class_names=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12,24,5)
        self.fc1 = nn.Linear(24*5*5,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3= nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.1426
Training epoch 1...
Loss: 1.7158
Training epoch 2...
Loss: 1.5187
Training epoch 3...
Loss: 1.3860
Training epoch 4...
Loss: 1.2832
Training epoch 5...
Loss: 1.2072
Training epoch 6...
Loss: 1.1458
Training epoch 7...
Loss: 1.0877
Training epoch 8...
Loss: 1.0408
Training epoch 9...
Loss: 0.9957
Training epoch 10...
Loss: 0.9574
Training epoch 11...
Loss: 0.9137
Training epoch 12...
Loss: 0.8831
Training epoch 13...
Loss: 0.8452
Training epoch 14...
Loss: 0.8125
Training epoch 15...
Loss: 0.7822
Training epoch 16...
Loss: 0.7531
Training epoch 17...
Loss: 0.7221
Training epoch 18...
Loss: 0.6979
Training epoch 19...
Loss: 0.6719
Training epoch 20...
Loss: 0.6480
Training epoch 21...
Loss: 0.6248
Training epoch 22...
Loss: 0.5993
Training epoch 23...
Loss: 0.5790
Training epoch 24...
Loss: 0.5580
Training epoch 25...
Loss: 0.5335
Training epoch 26...
Loss: 0.5112
Training epoch 27...
Loss: 0.4900
Training epoch 28...
Loss: 0.4742
Training epoch 29...
Los

In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')


In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\Dinesh kumar\AppData\Local\Temp\ipykernel_9608\1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [ ]:
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100* correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 68.58%


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image
image_paths = ['aero.jpg','dog.jpg','bird.jpeg']
images = [load_image(img) for img in image_paths]
net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: plane
Prediction: dog
Prediction: bird
